## Урок 3. Вебинар “Множество. Последовательность”

## Домашняя работа
### Математическая логика. Последовательность.

#### Математическая логика

Проверить любым способ, является ли данная логическая формула тавтологией:

1. $(A \cup B) \rightarrow(B \cup \overline{A})$

In [ ]:
"""
С помощью таблицы истинности получаем:
    A    B    (A ∪ B)    not(A)    (B ∪ not(A))    (A ∪ B) → (B ∪ not(A))
    0    0       0         1            1                   1
    0    1       1         1            1                   1
    1    0       1         0            0                   0
    1    1       1         0            1                   1
Таким образом формула не является тавтологией, так как не всегда принимает значение "истина" и имеет один из случаев когда она равна "ложь".
"""

2. $A \rightarrow(A \cup(\overline{B} \cap A))$

In [ ]:
"""
По закону поглащения для дизъюнкции в правой части формулы после упрощения получим A, то есть полученная формула:
𝐴 → 𝐴
Таким образом формула является тавтологией, так как формула будет всегда принимать "истина" при A = "Л" и при A = "И".
"""

Сформулируйте словесно высказывания (А: сегодня светит солнце; В: сегодня сыро; С: я поеду на дачу.):

3. $(\overline{A} \cup B) \rightarrow \overline{C}$

In [ ]:
# Сегодня НЕ светит солнце и сегодня сыро, поэтому я НЕ поеду на дачу.

4. $C \rightarrow(A \cup \overline{B})$

In [ ]:
# Если я поеду на дачу, значит сегодня светит солнце и сегодня НЕ сыро.

Пользуясь правилом построения противоположного высказывания, записать утверждения, противоположные следующим:

5. На любом курсе каждого факультета есть студенты, сдающие все экзамены на «отлично».

In [ ]:
# Есть курс и есть факультет, где все студенты не сдают все экзамены на "отлично".

6. Каждый студент философского факультета имеет друга, который умеет решать все логические задачи.

In [ ]:
# Есть студент не с философского факультета, который не имеет друга, не умеющего решать хотя бы одну логическую задачу.

7. В любом самолете на рейсе Вашингтон-Москва присутствует хотя бы один сотрудник силовых органов, в каждой пуговице одежды которого вмонтирован микрофон.

In [ ]:
# Есть самолет не на рейсе Вашингтон-Москва, где не присутствует любой сотрудник силовых органов, хотя бы в одной пуговице которого не вмонтирован микрофон.

#### Множества. Предел последовательности

Представьте в виде несократимой рациональной дроби:

8. 0.(216)

In [18]:
"""
a = 0.(216)
1000*a = 1000 * 0.(216) = 216.(216) = 216 + 0.(216) = 216 + a
1000*a - a = 216
999*a = 216
a = 216/999
a = 8/37 # сокращение на 27
"""
a = 8 / 37
a

0.21621621621621623

9. 1.0(01)

10. Представьте 1 в виде суммы трех рациональных дробей с разными знаменателями и числителем равным 1.

In [14]:
a = 1/2 + 1/3 + 1/6
a

0.9999999999999999

11*. Тоже задание, только в виде суммы шести дробей.

12. Найдите значение предела:
$$
\lim _{n \rightarrow \infty}\left(\frac{1}{1 \cdot 2}+\frac{1}{2 \cdot 3}+\frac{1}{3 \cdot 4}+\cdots+\frac{1}{(n-1) \cdot n}\right)
$$

#### Предел последовательности

13. Пользуясь критерием Коши, докажите сходимость последовательности: $$
a_{n}=\frac{\sin 1}{2}+\frac{\sin 2}{2^{2}}+\frac{\sin 3}{2^{3}}+\cdots+\frac{\sin n}{2^{n}}
$$ * Какой член последовательности можно взять в качестве предела с точностью $
\varepsilon=10^{-7}
$

14*. Пользуясь критерием Коши, докажите расходимость последовательности:
$$
b_{n}=1+\frac{1}{2}+\frac{1}{3}+\cdots+\frac{1}{n}
$$